In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

#Display DataFrame
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
FourBedroom_avg_price = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_sales where bedrooms == 4 group by YEAR(date) ORDER BY YEAR(date) desc")
FourBedroom_avg_price.show()



+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
ThreeBedroom_ThreeBathroom_avg_price = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_sales where bedrooms == 3 and bathrooms == 3 group by YEAR(date) ORDER BY YEAR(date) desc")
ThreeBedroom_ThreeBathroom_avg_price.show()



+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           292725.69|      2022|
|           294211.46|      2021|
|           294204.16|      2020|
|           287287.82|      2019|
+--------------------+----------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
ThreeBedroom_ThreeBathroom_avg_price = spark.sql("SELECT ROUND(AVG(price),2), date_built from home_sales where bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 group by date_built ORDER BY date_built desc")
ThreeBedroom_ThreeBathroom_avg_price.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
|           307539.97|      2012|
|           276553.81|      2011|
|           285010.22|      2010|
+--------------------+----------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_avg_price = spark.sql("SELECT ROUND(AVG(price),2), view from home_sales where price >= 350000 group by view ORDER BY view desc")
view_avg_price.show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           1026669.5| 100|
|          1061201.42|  99|
|          1053739.33|  98|
|          1129040.15|  97|
|          1017815.92|  96|
|           1054325.6|  95|
|           1033536.2|  94|
|          1026006.06|  93|
|           970402.55|  92|
|          1137372.73|  91|
|          1062654.16|  90|
|          1107839.15|  89|
|          1031719.35|  88|
|           1072285.2|  87|
|          1070444.25|  86|
|          1056336.74|  85|
|          1117233.13|  84|
|          1033965.93|  83|
|           1063498.0|  82|
|          1053472.79|  81|
+--------------------+----+
only showing top 20 rows

--- 0.6939995288848877 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')



In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

view_avg_price = spark.sql("SELECT ROUND(AVG(price),2), view from home_sales where price >= 350000 group by view ORDER BY view desc")
view_avg_price.show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           1026669.5| 100|
|          1061201.42|  99|
|          1053739.33|  98|
|          1129040.15|  97|
|          1017815.92|  96|
|           1054325.6|  95|
|           1033536.2|  94|
|          1026006.06|  93|
|           970402.55|  92|
|          1137372.73|  91|
|          1062654.16|  90|
|          1107839.15|  89|
|          1031719.35|  88|
|           1072285.2|  87|
|          1070444.25|  86|
|          1056336.74|  85|
|          1117233.13|  84|
|          1033965.93|  83|
|           1063498.0|  82|
|          1053472.79|  81|
+--------------------+----+
only showing top 20 rows

--- 0.9409699440002441 seconds ---


In [12]:
# Check DataFrame schema
home_sales_df.printSchema()

# Verify the existence of 'date_built' column
if 'date_built' in home_sales_df.columns:
    print("Column 'date_built' exists.")
else:
    print("Column 'date_built' does not exist in the DataFrame.")

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)

Column 'date_built' exists.


In [19]:
from pyspark.sql import SparkSession
import sys
import os


spark = SparkSession.builder.getOrCreate()
print("Spark Version:", spark.version)


Spark Version: 3.4.1


In [20]:
import py4j

print("Py4J Version:", py4j.__version__)


Py4J Version: 0.10.9.7


In [21]:
import subprocess

def get_java_version():
    try:
        # Run 'java -version' command and capture the output
        result = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT, text=True)
        lines = result.strip().split('\n')
        
        # Extract the first line containing the version info
        version_line = lines[0]

        # Extract the version number from the line (example: "openjdk version '11.0.12'")
        version = version_line.split(' ')[2].replace('\"', '')

        return version
    except subprocess.CalledProcessError:
        return "Java version not found."
    except Exception as e:
        return f"Error occurred: {e}"

# Call the function to get the Java version
java_version = get_java_version()
print("Java Version:", java_version)


Java Version: 1.8.0_25


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data and write out the results as parquet files.
home_sales_df.write.mode('overwrite').partitionBy('date_built').parquet('home_sales_parquet')


Py4JJavaError: An error occurred while calling o64.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fil

In [16]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

